 <font size="20"> Covid-19 Data Visualisation </font>

In [77]:
#importing libraries
from __future__ import division
import pycountry
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() 
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.express as px
import folium
from mpl_toolkits.basemap import Basemap
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from IPython.html import widgets
from IPython.display import display


<img src="COVID-19.png"/>'


In [78]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.

Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.

The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face. 

The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).

At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available.


In [84]:
def replaceCase(dataframe):
    i = 0
    for s in dataframe['Cases'].values: 
        s = str(s)
        s = s.replace('+', '')
        s = s.replace(',', '')
        s = int(float(s))
        dataframe['Cases'].values[i] = s
        i = i + 1

In [85]:
def replaceDeaths(dataframe):
    i = 0 
    for s in dataframe['Deaths'].values: 
        s = str(s)
        s = s.replace('+', '')
        s = s.replace(',', '')
        s = int(float(s))
        dataframe['Deaths'].values[i] = s
        i = i + 1


In [91]:
#dataframe = pd.read_csv('feb.csv')
url = 'https://raw.githubusercontent.com/ddevlin678/covid-19/master/feb.csv'
dataframe = pd.read_csv(url, error_bad_lines=False)
dataframe['Date'] =  pd.to_datetime(dataframe['Date'])
replaceCase(dataframe)
replaceDeaths(dataframe)
FebDataByCountry = dataframe.groupby('Country')
FebCountries = FebDataByCountry.groups.keys()

In [90]:
#dataframe11 = pd.read_csv('mar.csv')
url = 'https://raw.githubusercontent.com/ddevlin678/covid-19/master/mar.csv'
dataframe11 = pd.read_csv(url, error_bad_lines=False)
dataframe11['Date'] =  pd.to_datetime(dataframe11['Date'])
replaceCase(dataframe11)
replaceDeaths(dataframe11)
MarDataByCountry = dataframe11.groupby('Country')
MarCountries = MarDataByCountry.groups.keys()

In [89]:
#dataframe2 = pd.read_csv('april.csv')
url = 'https://raw.githubusercontent.com/ddevlin678/covid-19/master/april.csv'
dataframe2 = pd.read_csv(url, error_bad_lines=False)
dataframe2['Date'] =  pd.to_datetime(dataframe2['Date'])
replaceCase(dataframe2)
replaceDeaths(dataframe2)
AprilDataByCountry = dataframe2.groupby('Country')
AprilCountries = AprilDataByCountry.groups.keys()

In [92]:
#dataframe3 = pd.read_csv('may.csv')
url = 'https://raw.githubusercontent.com/ddevlin678/covid-19/master/april.csv'
dataframe3 = pd.read_csv(url, error_bad_lines=False)
dataframe3['Date'] =  pd.to_datetime(dataframe3['Date'])
replaceCase(dataframe3)
replaceDeaths(dataframe3)
MayDataByCountry = dataframe3.groupby('Country')
MayCountries = MayDataByCountry.groups.keys()

In [93]:
completeData = pd.concat([dataframe,dataframe11,dataframe2,dataframe3])
completeData['Date'] =  pd.to_datetime(completeData['Date'])
replaceCase(completeData)
replaceDeaths(completeData)
completeData = completeData.groupby('Country')
AllCountries = completeData.groups.keys()

In [94]:
dataComplete = pd.concat([dataframe,dataframe11,dataframe2,dataframe3])

In [95]:
#worldmap = pd.read_csv('worldmap.csv')
url = 'https://raw.githubusercontent.com/ddevlin678/covid-19/master/worldmap.csv'
worldmap = pd.read_csv(url, error_bad_lines=False)

In [56]:
objectdate = dataComplete['Date'].astype(str)

In [86]:
list_countries = dataComplete['Country'].unique().tolist()
iso = {}
for x in list_countries:
    try:
        data = pycountry.countries.search_fuzzy(x)
        iso.update({x: data[0].alpha_3})  
    except:
        iso.update({x: data[0].alpha_3})
for k, v in iso.items():
    dataComplete.loc[(dataComplete.Country == k), 'alpha3'] = v

In [18]:
months = ({'Feburary' :FebDataByCountry, 'March':MarDataByCountry, 'April':AprilDataByCountry, 'May':MayDataByCountry, 'All': completeData})

In [19]:
columns_names = ['Country','Total Cases', 'Cases', 'Total Deaths', 'Deaths', 'Total Recovered','Serious', 'Tested','Active Cases' ,'Date']

In [20]:
def LockdownSuccess(country):
    name = country['Country'].iloc[0]
    try:
        LockdownStartDate = pd.to_datetime(getDates(country)[0])
        LockdownEndDate = pd.to_datetime(getDates(country)[1])
    except:
        LockdownStartDate = ''
        LockdownEndDate = ''
    maxDeaths = country['Deaths'].max() 
    pos1 = [i for i ,j in enumerate(country['Deaths']) if j ==max(country['Deaths'])]
    d1 = country.iloc[pos1[0]]['Date']
    x = [0.2, 80]
    my_list = country['Deaths'].tolist()
    my_list = my_list[pos1[-1]:]
    result = [my_list[i:i+3] for i in range(len(my_list)-2) if all(i < int(maxDeaths*x[0]) for i in my_list[i:i+3])]
    if not result: 
        x = [0.3, 70]
        result = [my_list[i:i+3] for i in range(len(my_list)-2) if all(i < int(maxDeaths*x[0]) for i in my_list[i:i+3])]
    if not result: 
        x = [0.4, 60]
        result = [my_list[i:i+3] for i in range(len(my_list)-2) if all(i < int(maxDeaths*x[0]) for i in my_list[i:i+3])] 
    try: 
        pos2 = [i for i ,j in enumerate(country['Deaths']) if j ==result[0][-1]]
    except:
        pos2 = 0 
    if pos2 != 0: 
        d2 = country.iloc[pos2[-1]]['Date']
        delta = d2 - d1
        #print(delta.days,"Days to reduce deaths per day by", x[1],"%")
        return(delta.days,"Days to reduce daily deaths for 3 consecutive days by", x[1],"%")

In [21]:
def getDates(country):
    name = country['Country'].iloc[0]
    LockdownStartDate = LockdownDateDF.loc[name].loc['StartDate']
    LockdownEndDate = LockdownDateDF.loc[name].loc['EndDate']
    return (LockdownStartDate, LockdownEndDate)

In [22]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-application")

def geolocate(country):
    country1 = country['Country'].iloc[0].replace("_"," ")
    try:
        loc = geolocator.geocode(country1, timeout=10000)
        return (loc.latitude, loc.longitude, country1)
            
    except Exception as e:
        return ("Exception raised: {}".format(e))

In [23]:
def get_stats(country):
    try: 
        d = {'Cases': int(country['Cases'].sum()), 'Deaths': int(country['Deaths'].sum()), 
             'Max Deaths': int(country['Deaths'].max()),
             'Recovered':country['Total Recovered'].sum(),
              'Serious':country['Serious'].sum()}
        wi = pd.DataFrame([d])
        return (wi.to_string(index=False))
    except:
        #print("Exception raised: {}".format(e))
        d = {'Cases': int(country['Cases'].sum()), 'Deaths': int(country['Deaths'].sum()), 
             'Max Deaths': int(country['Deaths'].max()),
             'Recovered':0,
              'Serious':0}
        wi = pd.DataFrame([d])
        
        return (wi.to_string(index=False))
        return #wi #pd.DataFrame([d])


In [24]:
def plotting(country):
    countryName = country['Country'].iloc[0].replace("_"," ")
    x = country['Date']
    y1 = list(country['Cases'])
    y2 = list(country['Deaths'])
    fig, ax1 = plt.subplots(figsize=(14,5))
    fig.suptitle(countryName, fontsize=20)
    ax2 = ax1.twinx()
    ax1.plot(x, y1, 'g-')
    ax2.plot(x, y2, 'r-')
    ax1.set_xticks(ax1.get_xticks()[::2])
    ax1.set_xlabel('Dates',fontsize=15)
    ax1.set_ylabel('Cases', color='g',fontsize=15)
    ax2.set_ylabel('Deaths', color='r',fontsize=15)
    
    plt.xticks(rotation=90)
    plt.show(block=False)
plt.show()

In [25]:
#creating array of data to append to each dataset
from datetime import date, timedelta
mayx = []
sdate = date(2020, 5, 1)   # start date
edate = date(2020, 5, 27)   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    mayx.append(day)

<font size="5">Select a country and month to view deaths and cases</font>

In [76]:
def select_month(Month):
    countries.options = months[Month].groups.keys() 
    return Month   
def select_country(Country, Month):
    countryName = months[select_month(Month)].get_group(Country)
    plotting(countryName)
    #display(HTML(get_stats(countryName).to_html()))
    print(get_stats(countryName))
    print(LockdownSuccess(countryName))
    
scW = widgets.Select(options=(months.keys()))
init = scW.value
countries = widgets.Select(options=months[init].groups.keys())
j = widgets.interactive(select_country, Country=countries, Month = scW)
i = widgets.interactive(select_month, Month=scW)
#display(i)
display(j)

interactive(children=(Select(description='Country', options=('Afghanistan', 'Algeria', 'Australia', 'Austria',…

In [75]:
def rescale(deaths, new_min = 1, new_max = 20):
    Scaled_Deaths = []
    old_min, old_max = min(deaths), max(deaths)

    for v in deaths:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        Scaled_Deaths.append(new_v)

    return Scaled_Deaths

<font size=5> World map of countries affected by Covid-19</font> 

In [74]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)
for i in range(0,len(worldmap)):
    folium.Circle(
        location=[worldmap['lat'][i], worldmap['lon'][i]],
        fill=True,
        radius=(int((np.log(worldmap['cases'].iloc[i]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+worldmap['country'].iloc[i] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(worldmap['cases'].iloc[i])+"</li>"+
                        "<li>Deaths:   "+str(worldmap['deaths'].iloc[i])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(worldmap['deaths'].iloc[i]/(worldmap['cases'].iloc[i]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map

<font size=5> Countries most affected</font>

In [73]:
def chart(countries, attribute):
    df = worldmap.sort_values(attribute)
    fig = px.scatter(df.head(countries), x=df['country'], y=df[attribute], size=df[attribute], color=df['country'],
               hover_name=df['country'], size_max=60)
    fig.update_layout(
    title=str(countries) +" Countries with the most "+ attribute,
    xaxis_title="Countries",
    yaxis_title="Confirmed "+ attribute ,
    width = 700
    )
    fig.show();
interact(chart, countries=(1,25,1), attribute=worldmap.columns.values[5:]);

interactive(children=(IntSlider(value=13, description='countries', max=25, min=1), Dropdown(description='attri…

<font size=5> Timelapse of figures</font>

In [72]:
def chart1(Timelapse_Of):
    fig = px.choropleth(data_frame = dataComplete,
                    locations= "alpha3",
                    color= Timelapse_Of,  # value in column 'Confirmed' determines color
                    hover_name= "Country",
                    color_continuous_scale= "RdYlGn",  #  color scale red, yellow green
                    animation_frame= objectdate,
                    projection="natural earth")
    fig.update_layout(
    title=str(Timelapse_Of),
    width = 700
    )
    fig.show();
interact(chart1,Timelapse_Of=dataComplete[['Total Cases', 'Total Deaths', 'Total Recovered']]);

interactive(children=(Dropdown(description='Timelapse_Of', options=('Total Cases', 'Total Deaths', 'Total Reco…